### 크레딧 카드 사용 기록

#### 필요한 패키지를 불러온다

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

#### 데이터를 불러와서 살펴 본다

In [ ]:
os.chdir(r"C:\Users\sycha\Documents\교육사업\데이터 2019\핀인테크\20190722_아시아 경제\data")
df = pd.read_csv('data_credit.csv', header='infer',encoding='latin1')

In [ ]:
df.shape

In [ ]:
df.head(5)

#### 데이터를 클리닝하고 결측치 처리를 한다

In [ ]:
# 결측치가 분명히 있는 것 같으니 컬럼별 집계해 본다.
nas = df.isnull()
print(nas.sum(axis=0))

In [ ]:
# installments 변수의 도수분포를 본다.
df['installments'].value_counts()

In [ ]:
# 일시불이 NA로 처리 되었으니 결측치를 1로 채운다.
df = df.fillna(value=1)
df.head(5)

In [ ]:
# 다시 installments 변수의 도수분포를 본다.
df['installments'].value_counts()

In [ ]:
# 일단 date와 time은 사용하지 않겠으니 제거한다
df = df.drop(columns=['date','time'])
df.head(5)

#### 개개 변수를 다양한 방법으로 요약하고 시각화 한다

In [ ]:
#
# store_id의 도수의 히스토그램.
#
freq1 = df['store_id'].value_counts()
sns.distplot(freq1.values,bins=50)
plt.show()

In [ ]:
# 큰 skew를 보이므로 log 변환해 본다.
freq1b = np.log(freq1+1)
sns.distplot(freq1b.values,bins=30)
plt.show()

In [ ]:
print(np.median(freq1))

In [ ]:
# Top sore id를 본다.
freq1.sort_values(ascending=False)[:10]

In [ ]:
#
# card_id의 도수의 히스토그램.
#
freq2 = df['card_id'].value_counts()
sns.distplot(freq2.values,bins=50,color="green")
plt.show()

In [ ]:
# 큰 skew를 보이므로 log 변환해 본다.
freq2b = np.log(freq2+1)
sns.distplot(freq2b.values,bins=20,color="green",kde=False)
plt.show()

In [ ]:
# Top 10 card id를 본다.
freq2.sort_values(ascending=False)[:10]

In [ ]:
#
# amount의 히스토그램.
# 
sns.distplot(df.amount,bins=10,kde=False)
plt.show()

In [ ]:
#
# 실제 구매 or Cancel.
#
ser = df.amount > 0
np.round(ser.mean(),4)             # 실제 구매의 퍼센티지.

In [ ]:
#
# installments의 막대그림.
# 
freq3 = df['installments'].value_counts()
print("installments의 가짓수 = {}".format((freq3.index).nunique()))

In [ ]:
# Seaborn의 countplot이 집계하여 보여준다.
sns.countplot(data=df, x='installments')
plt.show()

In [ ]:
# 1이 너무나 많으니 제거하고 다시 시각화 해본다.
sns.countplot(data=df[df.installments != 1], x='installments')
plt.show()

In [ ]:
#
# days_of_week의 횟수 막대그림.
# 
freq4 = df['days_of_week'].value_counts()
print("days of week의 가짓수 = {}".format((freq4.index).nunique()))

In [ ]:
# Seaborn의 countplot이 집계하여 보여준다.
sns.countplot(data=df, x='days_of_week')
plt.show()

In [ ]:
#
# holiday의 막대그림.
# 
sns.countplot(data=df, x='holyday')
plt.show()

#### 일변량 요약과 시각화

In [ ]:
# 휴일 여부별 평균 사용 금액.
df.groupby('holyday')['amount'].mean()

In [ ]:
# 요일별 평균 결제 금액.
ser1 = df.groupby('days_of_week')['amount'].mean()
ser1

In [ ]:
sns.barplot(ser1.index, ser1.values)
plt.show()

In [ ]:
# store_id 별 평균 결제 금액.
ser2 = df.groupby('store_id')['amount'].mean()
sns.distplot(ser2).set_title('Mean Amount by Store')
plt.show()
sns.distplot(np.log(ser2+1),color='orange').set_title('Mean Amount by Store [Logged]')
plt.show()

In [ ]:
# store_id 별 Total 결제 금액.
ser2b = df.groupby('store_id')['amount'].sum()
sns.distplot(ser2b).set_title('Total Amount by Store')
plt.show()
sns.distplot(np.log(ser2b+1),color='orange').set_title('Total Amount by Store [Logged]')
plt.show()

In [ ]:
# card_id 별 평균 결제 금액.
ser3 = df.groupby('card_id')['amount'].mean()
sns.distplot(ser3, kde=False).set_title('Mean Amount by Card')
plt.show()

In [ ]:
# card_id 별 Total 결제 금액.
ser3b = df.groupby('card_id')['amount'].mean()
sns.distplot(ser3b, kde=False).set_title('Total Amount by Card')
plt.show()

#### 분류

In [ ]:
#
# store_id 별 Total 결제 금액에 의한 3 등급 분류.
#
ser2c = pd.qcut(ser2b,3,labels=['Low', 'Medium', 'High'])
combo2 = pd.DataFrame({"amount":ser2b.values, "category":ser2c.values})
combo2.head(5)

In [ ]:
# 등급별 평균.
combo2.groupby("category")["amount"].mean()

In [ ]:
#
# card_id 별 Total 결제 금액에 의한 3 등급 분류.
#
ser3c = pd.qcut(ser3b,3,labels=['Low', 'Medium', 'High'])
combo3 = pd.DataFrame({"amount":ser3b.values, "category":ser3c.values})
combo3.head(5)

In [ ]:
# 등급별 평균.
combo3.groupby("category")["amount"].mean()

#### 이변량 요약과 시각화

In [ ]:
# installment 대 amount
ser = df.groupby('installments')['amount'].mean()
sns.jointplot(ser.index, ser.values, kind='reg')  # installments 로그변환.
plt.show()

In [ ]:
# 거래 횟수 대 총 거래 금액
sera = np.log(df['store_id'].value_counts()+1)             # 로그변환.
serb = np.log(df.groupby('store_id')['amount'].sum()+1)    # 로그변환.

In [ ]:
df2 = pd.concat([sera,serb],axis=1)
df2.columns = ['Frequency', 'Amount']      # 컬럼 명칭을 바로 쓴다.

In [ ]:
sns.jointplot(x='Frequency',y='Amount',data=df2,kind="reg", size=10)
plt.show()